In [1]:
from cherche import data, retrieve
from transformers import pipeline
import pandas as pd
from elasticsearch import Elasticsearch
from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
import time

In [2]:
caminho_query = "dados-conle-anonimizado-assunto-notnull - dados-conle-anonimizado-assunto-notnull.tsv"

# Base de dados câmara dos deputados

In [3]:
df_assunto= pd.read_table(caminho_query, delimiter=";")

In [4]:
arr_assunto = df_assunto.to_numpy()
y,X = arr_assunto[:,0],arr_assunto[:,1]
y = [i.strip() for i in y]

In [5]:
def verificar(y,top_n):
    for d in top_n:
        if y == d:
            return 1
    return 0

In [6]:
def avaliacaoRecall(isPreprocess):

    quant_encontrado=0
    quant_relevante =0
    antes = time.time()
    for l,x in zip(y,X):

        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        scores = retriever(tokenized_query3)
        top_n = [d["name"] for d in scores]  

        
        quant_relevante+=1
        quant_encontrado+=verificar(l,top_n)
    
    recall = quant_encontrado / quant_relevante
    depois = time.time()
    duracao = depois - antes
    print("Duração: %f" %(duracao))
    print("Recall: "+str(recall))

In [7]:
def avaliacaoRR(isPreprocess):

    quant_encontrado=0
    for l,x in zip(y,X):

        tokenized_query3 = x                   
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        scores = retriever(tokenized_query3)
        top_n = scores[0]["name"]
        if top_n.strip() == l.strip():
            quant_encontrado+=1

    
    rr = quant_encontrado / 295
    print("RR: %f" % (rr))

In [8]:
def avaliacaoPrecision(isPreprocess,top_k=20):
    soma=0
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        scores = retriever(tokenized_query3)
        top_n = [d["name"] for d in scores] 
    
        encontrado=verificar(l,top_n)
        soma+= (encontrado / top_k)
    precision = soma / 295
    print("P@ %d: %f " % (top_k,precision))

In [9]:
def avaliacaoMAP(isPreprocess,top_k=20):
    l_v = list()
    labels_nsL = list()
    for l,x in zip(y,X):

        tokenized_query3 = x
        if isPreprocess:
            tokenized_query3 = preprocess(x)
    
        scores = retriever(tokenized_query3)
        top_n = [d["name"] for d in scores]
        l_v.append(l)
        labels_nsL.append(top_n)
        
    denominador = 0
    encontrou=False
    soma=0
    for Y,x in zip(l_v,labels_nsL):
        calc = 0
        quant=0
        for k in x:
            if str(k).strip() == Y.strip():
                quant=1
                encontrou=True
            denominador+=1
            calc=quant/denominador
            if encontrou:
                calc = calc * 1
            else:
                calc = calc * 0
            soma+=calc
            encontrou = False
    MAP = soma /295
    print("MAP@ %d: %f" % (top_k,MAP))

In [10]:
class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word

In [11]:
class RSLP_S:
    def __plural_reduction(self, word):
        excep = ["lápis","cais","mais","crúcis","biquínis","pois","depois","dois","leis" ]
        excep_s = ["aliás","pires","lápis","cais","mais","mas","menos", "férias","fezes","pêsames","crúcis","gás", "atrás","moisés","através","convés","ês", "país","após","ambas","ambos","messias"]

        len_word = len(word)
        new_word = list(word)

        if len_word >= 3:
            if new_word[-1] == 's' and new_word[-2] == 'n':
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'õ':
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return  sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'ã':
                if word == 'mães':
                    word = word[:-1]
                    return word
                else:
                    new_word[-2] = 'o'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'a':
                if word != 'cais' and word != 'mais':
                    new_word[-2] = 'l'
                    sing = "".join(new_word)
                    sing = sing[:-1]
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'é':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'e':
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i' and new_word[-3] == 'ó':
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

            if new_word[-1] == 's' and new_word[-2] == 'i':
                if word not in excep:
                    new_word[-1] = 'l'
                    sing = "".join(new_word)
                    return sing

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'l':
                word = word[:-2]
                return word

            if new_word[-1] == 's' and new_word[-2] == 'e' and new_word[-3] == 'r':
                word = word[:-2]
                return word

            if new_word[-1] == 's':
                if word not in excep_s:
                    word = word[:-1]

        return word

    def stem(self, word):
        word = self.__plural_reduction(word)

        return word

In [12]:
es = Elasticsearch(hosts="http://localhost:9200", http_auth=('elastic', 'TlHbFKwz9YMAprROSrgA'),timeout=30)

# Pré processamento

## 1- Sem pré processamento

In [13]:
indice = "sem_preprocessamento_img"

## Avaliação

#### TOP K 5

In [14]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [15]:
avaliacaoRecall(False)

Duração: 29.567302
Recall: 0.6101694915254238


In [16]:
avaliacaoPrecision(False,5)

P@ 5: 0.122034 


In [17]:
avaliacaoMAP(False,5)

MAP@ 5: 0.004146


In [18]:
avaliacaoRR(False)

RR: 0.389831


#### TOP K 10

In [19]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [20]:
avaliacaoRecall(False)

Duração: 24.584935
Recall: 0.6677966101694915


In [21]:
avaliacaoPrecision(False,10)

P@ 10: 0.066780 


In [22]:
avaliacaoMAP(False,10)

MAP@ 10: 0.003050


In [23]:
avaliacaoRR(False)

RR: 0.389831


#### TOP K 20

In [24]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [25]:
avaliacaoRecall(False)

Duração: 33.020497
Recall: 0.7016949152542373


In [26]:
avaliacaoPrecision(False,20)

P@ 20: 0.035085 


In [27]:
avaliacaoMAP(False,20)

MAP@ 20: 0.002415


In [28]:
avaliacaoRR(False)

RR: 0.389831


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [29]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [30]:
indice = "preprocessamento_5_img"

## Avaliação

#### TOP K 5

In [31]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [33]:
avaliacaoRecall(True)

Duração: 19.522175
Recall: 0.6135593220338983


In [34]:
avaliacaoPrecision(True,5)

P@ 5: 0.122712 


In [35]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003567


In [36]:
avaliacaoRR(True)

RR: 0.379661


#### TOP K 10

In [37]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [38]:
avaliacaoRecall(True)

Duração: 14.222872
Recall: 0.6644067796610169


In [39]:
avaliacaoPrecision(True,10)

P@ 10: 0.066441 


In [40]:
avaliacaoMAP(True,10)

MAP@ 10: 0.002488


In [41]:
avaliacaoRR(True)

RR: 0.379661


#### TOP K 20

In [42]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [43]:
avaliacaoRecall(True)

Duração: 16.391733
Recall: 0.7050847457627119


In [44]:
avaliacaoPrecision(True,20)

P@ 20: 0.035254 


In [45]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001845


In [46]:
avaliacaoRR(True)

RR: 0.379661


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [47]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [48]:
indice = "preprocessamento_8_img"

## Avaliação

#### TOP K 5

In [49]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [50]:
avaliacaoRecall(True)

Duração: 21.799000
Recall: 0.6101694915254238


In [51]:
avaliacaoPrecision(True,5)

P@ 5: 0.122034 


In [52]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003631


In [53]:
avaliacaoRR(True)

RR: 0.396610


#### TOP K 10

In [54]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [55]:
avaliacaoRecall(True)

Duração: 14.311227
Recall: 0.6644067796610169


In [56]:
avaliacaoPrecision(True,10)

P@ 10: 0.066441 


In [57]:
avaliacaoMAP(True,10)

MAP@ 10: 0.003004


In [58]:
avaliacaoRR(True)

RR: 0.396610


#### TOP K 20

In [59]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [60]:
avaliacaoRecall(True)

Duração: 15.532029
Recall: 0.7152542372881356


In [61]:
avaliacaoPrecision(True,20)

P@ 20: 0.035763 


In [62]:
avaliacaoMAP(True,20)

MAP@ 20: 0.002419


In [63]:
avaliacaoRR(True)

RR: 0.396610


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [64]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [65]:
indice = "preprocessamento_9_img"

## Avaliação

#### TOP K 5

In [66]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [67]:
avaliacaoRecall(True)

Duração: 15.325752
Recall: 0.6101694915254238


In [68]:
avaliacaoPrecision(True,5)

P@ 5: 0.122034 


In [69]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003485


In [70]:
avaliacaoRR(True)

RR: 0.389831


#### TOP K 10

In [71]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [72]:
avaliacaoRecall(True)

Duração: 11.046416
Recall: 0.6440677966101694


In [73]:
avaliacaoPrecision(True,10)

P@ 10: 0.064407 


In [74]:
avaliacaoMAP(True,10)

MAP@ 10: 0.002437


In [75]:
avaliacaoRR(True)

RR: 0.389831


#### TOP K 20

In [76]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [77]:
avaliacaoRecall(True)

Duração: 12.330267
Recall: 0.711864406779661


In [78]:
avaliacaoPrecision(True,20)

P@ 20: 0.035593 


In [79]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001872


In [80]:
avaliacaoRR(True)

RR: 0.389831


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [93]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [94]:
indice = "preprocessamento_18_img"

## Avaliação

#### TOP K 5

In [95]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [96]:
avaliacaoRecall(True)

Duração: 8.331251
Recall: 0.6169491525423729


In [97]:
avaliacaoPrecision(True,5)

P@ 5: 0.123390 


In [98]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002270


In [99]:
avaliacaoRR(True)

RR: 0.410169


#### TOP K 10

In [100]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [101]:
avaliacaoRecall(True)

Duração: 9.031117
Recall: 0.6711864406779661


In [102]:
avaliacaoPrecision(True,10)

P@ 10: 0.067119 


In [103]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001442


In [104]:
avaliacaoRR(True)

RR: 0.410169


#### TOP K 20

In [105]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [106]:
avaliacaoRecall(True)

Duração: 11.210618
Recall: 0.7220338983050848


In [107]:
avaliacaoPrecision(True,20)

P@ 20: 0.036102 


In [108]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001006


In [109]:
avaliacaoRR(True)

RR: 0.410169


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [110]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [111]:
indice = "preprocessamento_21_img"

## Avaliação

#### TOP K 5

In [112]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [113]:
avaliacaoRecall(True)

Duração: 7.200141
Recall: 0.6135593220338983


In [114]:
avaliacaoPrecision(True,5)

P@ 5: 0.122712 


In [115]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002393


In [116]:
avaliacaoRR(True)

RR: 0.406780


#### TOP K 10

In [117]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [118]:
avaliacaoRecall(True)

Duração: 7.314510
Recall: 0.6711864406779661


In [119]:
avaliacaoPrecision(True,10)

P@ 10: 0.067119 


In [120]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001433


In [121]:
avaliacaoRR(True)

RR: 0.406780


#### TOP K 20

In [122]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [123]:
avaliacaoRecall(True)

Duração: 9.858643
Recall: 0.7220338983050848


In [124]:
avaliacaoPrecision(True,20)

P@ 20: 0.036102 


In [125]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001074


In [126]:
avaliacaoRR(True)

RR: 0.406780


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP-S)

In [127]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [128]:
indice = "preprocessamento_22_img"

## Avaliação

#### TOP K 5

In [129]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [130]:
avaliacaoRecall(True)

Duração: 5.925020
Recall: 0.6169491525423729


In [131]:
avaliacaoPrecision(True,5)

P@ 5: 0.123390 


In [132]:
avaliacaoMAP(True,5)

MAP@ 5: 0.003034


In [133]:
avaliacaoRR(True)

RR: 0.372881


#### TOP K 10

In [134]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [135]:
avaliacaoRecall(True)

Duração: 6.366754
Recall: 0.6508474576271186


In [136]:
avaliacaoPrecision(True,10)

P@ 10: 0.065085 


In [137]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001988


In [138]:
avaliacaoRR(True)

RR: 0.372881


#### TOP K 20

In [139]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [140]:
avaliacaoRecall(True)

Duração: 6.244180
Recall: 0.7050847457627119


In [141]:
avaliacaoPrecision(True,20)

P@ 20: 0.035254 


In [142]:
avaliacaoMAP(True,20)

MAP@ 20: 0.001387


In [143]:
avaliacaoRR(True)

RR: 0.372881


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP-S) + unigram + bigram

In [144]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [145]:
indice = "preprocessamento_23_img"

## Avaliação

#### TOP K 5

In [146]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [147]:
avaliacaoRecall(True)

Duração: 6.661061
Recall: 0.6067796610169491


In [148]:
avaliacaoPrecision(True,5)

P@ 5: 0.121356 


In [149]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002423


In [150]:
avaliacaoRR(True)

RR: 0.410169


#### TOP K 10

In [151]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [152]:
avaliacaoRecall(True)

Duração: 6.990102
Recall: 0.6576271186440678


In [153]:
avaliacaoPrecision(True,10)

P@ 10: 0.065763 


In [154]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001344


In [155]:
avaliacaoRR(True)

RR: 0.410169


#### TOP K 20

In [156]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [157]:
avaliacaoRecall(True)

Duração: 7.762531
Recall: 0.711864406779661


In [158]:
avaliacaoPrecision(True,20)

P@ 20: 0.035593 


In [159]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000961


In [160]:
avaliacaoRR(True)

RR: 0.410169


# Indexacao

## 1- Sem pré processamento

In [161]:
indice = "sem_preprocessamento_indexacao"

## Avaliação

#### TOP K 5

In [162]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [163]:
avaliacaoRecall(False)

Duração: 3.402009
Recall: 0.2135593220338983


In [164]:
avaliacaoPrecision(False,5)

P@ 5: 0.042712 


In [165]:
avaliacaoMAP(False,5)

MAP@ 5: 0.000430


In [166]:
avaliacaoRR(False)

RR: 0.111864


#### TOP K 10

In [167]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [168]:
avaliacaoRecall(False)

Duração: 2.333836
Recall: 0.2677966101694915


In [169]:
avaliacaoPrecision(False,10)

P@ 10: 0.026780 


In [170]:
avaliacaoMAP(False,10)

MAP@ 10: 0.000404


In [171]:
avaliacaoRR(False)

RR: 0.111864


#### TOP K 20

In [172]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [173]:
avaliacaoRecall(False)

Duração: 2.393322
Recall: 0.30847457627118646


In [174]:
avaliacaoPrecision(False,20)

P@ 20: 0.015424 


In [175]:
avaliacaoMAP(False,20)

MAP@ 20: 0.000252


In [176]:
avaliacaoRR(False)

RR: 0.111864


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [177]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [178]:
indice = "preprocessamento_5_indexacao"

## Avaliação

#### TOP K 5

In [179]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [180]:
avaliacaoRecall(True)

Duração: 2.153448
Recall: 0.31864406779661014


In [181]:
avaliacaoPrecision(True,5)

P@ 5: 0.063729 


In [182]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001034


In [183]:
avaliacaoRR(True)

RR: 0.169492


#### TOP K 10

In [184]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [185]:
avaliacaoRecall(True)

Duração: 1.412678
Recall: 0.3525423728813559


In [186]:
avaliacaoPrecision(True,10)

P@ 10: 0.035254 


In [187]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000687


In [188]:
avaliacaoRR(True)

RR: 0.169492


#### TOP K 20

In [189]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [190]:
avaliacaoRecall(True)

Duração: 1.358128
Recall: 0.42033898305084744


In [191]:
avaliacaoPrecision(True,20)

P@ 20: 0.021017 


In [192]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000406


In [193]:
avaliacaoRR(True)

RR: 0.169492


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [194]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [195]:
indice = "preprocessamento_8_indexacao"

## Avaliação

#### TOP K 5

In [196]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [197]:
avaliacaoRecall(True)

Duração: 3.365621
Recall: 0.4033898305084746


In [198]:
avaliacaoPrecision(True,5)

P@ 5: 0.080678 


In [199]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001494


In [200]:
avaliacaoRR(True)

RR: 0.230508


#### TOP K 10

In [201]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [202]:
avaliacaoRecall(True)

Duração: 3.564423
Recall: 0.4576271186440678


In [203]:
avaliacaoPrecision(True,10)

P@ 10: 0.045763 


In [204]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000855


In [205]:
avaliacaoRR(True)

RR: 0.230508


#### TOP K 20

In [206]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [207]:
avaliacaoRecall(True)

Duração: 2.120884
Recall: 0.5050847457627119


In [208]:
avaliacaoPrecision(True,20)

P@ 20: 0.025254 


In [209]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000489


In [210]:
avaliacaoRR(True)

RR: 0.230508


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [211]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [212]:
indice = "preprocessamento_9_indexacao"

## Avaliação

#### TOP K 5

In [213]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [214]:
avaliacaoRecall(True)

Duração: 2.364712
Recall: 0.42033898305084744


In [215]:
avaliacaoPrecision(True,5)

P@ 5: 0.084068 


In [216]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001295


In [217]:
avaliacaoRR(True)

RR: 0.240678


#### TOP K 10

In [218]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [219]:
avaliacaoRecall(True)

Duração: 1.567612
Recall: 0.4542372881355932


In [220]:
avaliacaoPrecision(True,10)

P@ 10: 0.045424 


In [221]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000897


In [222]:
avaliacaoRR(True)

RR: 0.240678


#### TOP K 20

In [223]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [224]:
avaliacaoRecall(True)

Duração: 1.612820
Recall: 0.511864406779661


In [225]:
avaliacaoPrecision(True,20)

P@ 20: 0.025593 


In [226]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000493


In [227]:
avaliacaoRR(True)

RR: 0.240678


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [228]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [229]:
indice = "preprocessamento_18_indexacao"

## Avaliação

#### TOP K 5

In [230]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [231]:
avaliacaoRecall(True)

Duração: 3.931814
Recall: 0.3864406779661017


In [232]:
avaliacaoPrecision(True,5)

P@ 5: 0.077288 


In [233]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001236


In [234]:
avaliacaoRR(True)

RR: 0.220339


#### TOP K 10

In [235]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [236]:
avaliacaoRecall(True)

Duração: 2.368580
Recall: 0.4406779661016949


In [237]:
avaliacaoPrecision(True,10)

P@ 10: 0.044068 


In [238]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000795


In [239]:
avaliacaoRR(True)

RR: 0.220339


#### TOP K 20

In [240]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [241]:
avaliacaoRecall(True)

Duração: 2.468899
Recall: 0.4847457627118644


In [242]:
avaliacaoPrecision(True,20)

P@ 20: 0.024237 


In [243]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000466


In [244]:
avaliacaoRR(True)

RR: 0.220339


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [245]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [246]:
indice = "preprocessamento_21_indexacao"

## Avaliação

#### TOP K 5

In [247]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [248]:
avaliacaoRecall(True)

Duração: 3.296456
Recall: 0.38305084745762713


In [249]:
avaliacaoPrecision(True,5)

P@ 5: 0.076610 


In [250]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001144


In [251]:
avaliacaoRR(True)

RR: 0.210169


#### TOP K 10

In [252]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [253]:
avaliacaoRecall(True)

Duração: 2.248507
Recall: 0.43728813559322033


In [254]:
avaliacaoPrecision(True,10)

P@ 10: 0.043729 


In [255]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000783


In [256]:
avaliacaoRR(True)

RR: 0.210169


#### TOP K 20

In [257]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [258]:
avaliacaoRecall(True)

Duração: 2.011024
Recall: 0.4847457627118644


In [259]:
avaliacaoPrecision(True,20)

P@ 20: 0.024237 


In [260]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000462


In [261]:
avaliacaoRR(True)

RR: 0.210169


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP-S)

In [262]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [263]:
indice = "preprocessamento_22_indexacao"

## Avaliação

#### TOP K 5

In [264]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [265]:
avaliacaoRecall(True)

Duração: 2.270353
Recall: 0.4


In [266]:
avaliacaoPrecision(True,5)

P@ 5: 0.080000 


In [267]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001214


In [268]:
avaliacaoRR(True)

RR: 0.230508


#### TOP K 10

In [269]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [270]:
avaliacaoRecall(True)

Duração: 1.603706
Recall: 0.4406779661016949


In [271]:
avaliacaoPrecision(True,10)

P@ 10: 0.044068 


In [272]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000858


In [273]:
avaliacaoRR(True)

RR: 0.230508


#### TOP K 20

In [274]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [275]:
avaliacaoRecall(True)

Duração: 1.541616
Recall: 0.5050847457627119


In [276]:
avaliacaoPrecision(True,20)

P@ 20: 0.025254 


In [277]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000481


In [278]:
avaliacaoRR(True)

RR: 0.230508


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP-S) + unigram + bigram

In [279]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [280]:
indice = "preprocessamento_23_indexacao"

## Avaliação

#### TOP K 5

In [281]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [282]:
avaliacaoRecall(True)

Duração: 3.219137
Recall: 0.376271186440678


In [283]:
avaliacaoPrecision(True,5)

P@ 5: 0.075254 


In [284]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001367


In [285]:
avaliacaoRR(True)

RR: 0.210169


#### TOP K 10

In [286]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [287]:
avaliacaoRecall(True)

Duração: 1.935063
Recall: 0.4271186440677966


In [288]:
avaliacaoPrecision(True,10)

P@ 10: 0.042712 


In [289]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000791


In [290]:
avaliacaoRR(True)

RR: 0.210169


#### TOP K 20

In [291]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [292]:
avaliacaoRecall(True)

Duração: 1.847997
Recall: 0.4847457627118644


In [293]:
avaliacaoPrecision(True,20)

P@ 20: 0.024237 


In [294]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000469


In [295]:
avaliacaoRR(True)

RR: 0.210169


# Emenda

## 1- Sem pré processamento

In [296]:
indice = "sem_preprocessamento_emenda"

## Avaliação

#### TOP K 5

In [297]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [298]:
avaliacaoRecall(False)

Duração: 4.898598
Recall: 0.3864406779661017


In [299]:
avaliacaoPrecision(False,5)

P@ 5: 0.077288 


In [300]:
avaliacaoMAP(False,5)

MAP@ 5: 0.001689


In [301]:
avaliacaoRR(False)

RR: 0.244068


#### TOP K 10

In [302]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [303]:
avaliacaoRecall(False)

Duração: 3.709680
Recall: 0.43728813559322033


In [304]:
avaliacaoPrecision(False,10)

P@ 10: 0.043729 


In [305]:
avaliacaoMAP(False,10)

MAP@ 10: 0.000939


In [306]:
avaliacaoRR(False)

RR: 0.244068


#### TOP K 20

In [307]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [308]:
avaliacaoRecall(False)

Duração: 3.897678
Recall: 0.4711864406779661


In [309]:
avaliacaoPrecision(False,20)

P@ 20: 0.023559 


In [310]:
avaliacaoMAP(False,20)

MAP@ 20: 0.000493


In [311]:
avaliacaoRR(False)

RR: 0.244068


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [312]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [313]:
indice = "preprocessamento_5_emenda"

## Avaliação

#### TOP K 5

In [314]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [315]:
avaliacaoRecall(True)

Duração: 2.249341
Recall: 0.4


In [316]:
avaliacaoPrecision(True,5)

P@ 5: 0.080000 


In [317]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001732


In [318]:
avaliacaoRR(True)

RR: 0.267797


#### TOP K 10

In [319]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [320]:
avaliacaoRecall(True)

Duração: 1.512552
Recall: 0.4406779661016949


In [321]:
avaliacaoPrecision(True,10)

P@ 10: 0.044068 


In [322]:
avaliacaoMAP(True,10)

MAP@ 10: 0.000951


In [323]:
avaliacaoRR(True)

RR: 0.267797


#### TOP K 20

In [324]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [325]:
avaliacaoRecall(True)

Duração: 1.445605
Recall: 0.488135593220339


In [326]:
avaliacaoPrecision(True,20)

P@ 20: 0.024407 


In [327]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000500


In [328]:
avaliacaoRR(True)

RR: 0.267797


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [329]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [330]:
indice = "preprocessamento_8_emenda"

## Avaliação

#### TOP K 5

In [331]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [332]:
avaliacaoRecall(True)

Duração: 3.274497
Recall: 0.4576271186440678


In [333]:
avaliacaoPrecision(True,5)

P@ 5: 0.091525 


In [334]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001991


In [335]:
avaliacaoRR(True)

RR: 0.277966


#### TOP K 10

In [336]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [337]:
avaliacaoRecall(True)

Duração: 2.149607
Recall: 0.5050847457627119


In [338]:
avaliacaoPrecision(True,10)

P@ 10: 0.050508 


In [339]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001080


In [340]:
avaliacaoRR(True)

RR: 0.277966


#### TOP K 20

In [341]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [342]:
avaliacaoRecall(True)

Duração: 2.299304
Recall: 0.5457627118644067


In [343]:
avaliacaoPrecision(True,20)

P@ 20: 0.027288 


In [344]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000572


In [345]:
avaliacaoRR(True)

RR: 0.277966


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [346]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [347]:
indice = "preprocessamento_9_emenda"

## Avaliação

#### TOP K 5

In [348]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [349]:
avaliacaoRecall(True)

Duração: 2.664680
Recall: 0.45084745762711864


In [350]:
avaliacaoPrecision(True,5)

P@ 5: 0.090169 


In [351]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001937


In [352]:
avaliacaoRR(True)

RR: 0.288136


#### TOP K 10

In [353]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [354]:
avaliacaoRecall(True)

Duração: 1.693492
Recall: 0.5016949152542373


In [355]:
avaliacaoPrecision(True,10)

P@ 10: 0.050169 


In [356]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001075


In [357]:
avaliacaoRR(True)

RR: 0.288136


#### TOP K 20

In [358]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [359]:
avaliacaoRecall(True)

Duração: 1.716918
Recall: 0.5457627118644067


In [360]:
avaliacaoPrecision(True,20)

P@ 20: 0.027288 


In [361]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000569


In [362]:
avaliacaoRR(True)

RR: 0.288136


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [363]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [364]:
indice = "preprocessamento_18_emenda"

## Avaliação

#### TOP K 5

In [365]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [366]:
avaliacaoRecall(True)

Duração: 3.705877
Recall: 0.4542372881355932


In [367]:
avaliacaoPrecision(True,5)

P@ 5: 0.090847 


In [368]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001966


In [369]:
avaliacaoRR(True)

RR: 0.301695


#### TOP K 10

In [370]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [371]:
avaliacaoRecall(True)

Duração: 2.452944
Recall: 0.49830508474576274


In [372]:
avaliacaoPrecision(True,10)

P@ 10: 0.049831 


In [373]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001059


In [374]:
avaliacaoRR(True)

RR: 0.301695


#### TOP K 20

In [375]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [376]:
avaliacaoRecall(True)

Duração: 2.616044
Recall: 0.5322033898305085


In [377]:
avaliacaoPrecision(True,20)

P@ 20: 0.026610 


In [378]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000562


In [379]:
avaliacaoRR(True)

RR: 0.301695


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [380]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [381]:
indice = "preprocessamento_21_emenda"

## Avaliação

#### TOP K 5

In [382]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [383]:
avaliacaoRecall(True)

Duração: 3.712846
Recall: 0.4576271186440678


In [384]:
avaliacaoPrecision(True,5)

P@ 5: 0.091525 


In [385]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001965


In [386]:
avaliacaoRR(True)

RR: 0.301695


#### TOP K 10

In [387]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [388]:
avaliacaoRecall(True)

Duração: 2.279651
Recall: 0.5050847457627119


In [389]:
avaliacaoPrecision(True,10)

P@ 10: 0.050508 


In [390]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001074


In [391]:
avaliacaoRR(True)

RR: 0.301695


#### TOP K 20

In [392]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [393]:
avaliacaoRecall(True)

Duração: 2.182982
Recall: 0.5220338983050847


In [394]:
avaliacaoPrecision(True,20)

P@ 20: 0.026102 


In [395]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000559


In [396]:
avaliacaoRR(True)

RR: 0.301695


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP-S)

In [397]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [398]:
indice = "preprocessamento_22_emenda"

## Avaliação

#### TOP K 5

In [399]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [400]:
avaliacaoRecall(True)

Duração: 2.315923
Recall: 0.4271186440677966


In [401]:
avaliacaoPrecision(True,5)

P@ 5: 0.085424 


In [402]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001772


In [403]:
avaliacaoRR(True)

RR: 0.281356


#### TOP K 10

In [404]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [405]:
avaliacaoRecall(True)

Duração: 1.521955
Recall: 0.488135593220339


In [406]:
avaliacaoPrecision(True,10)

P@ 10: 0.048814 


In [407]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001055


In [408]:
avaliacaoRR(True)

RR: 0.281356


#### TOP K 20

In [409]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [410]:
avaliacaoRecall(True)

Duração: 1.577729
Recall: 0.5186440677966102


In [411]:
avaliacaoPrecision(True,20)

P@ 20: 0.025932 


In [412]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000554


In [413]:
avaliacaoRR(True)

RR: 0.281356


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP-S) + unigram + bigram

In [414]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [415]:
indice = "preprocessamento_23_emenda"

## Avaliação

#### TOP K 5

In [416]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [417]:
avaliacaoRecall(True)

Duração: 3.965345
Recall: 0.45084745762711864


In [418]:
avaliacaoPrecision(True,5)

P@ 5: 0.090169 


In [419]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001821


In [420]:
avaliacaoRR(True)

RR: 0.294915


#### TOP K 10

In [421]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [422]:
avaliacaoRecall(True)

Duração: 2.030308
Recall: 0.49491525423728816


In [423]:
avaliacaoPrecision(True,10)

P@ 10: 0.049492 


In [424]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001057


In [425]:
avaliacaoRR(True)

RR: 0.294915


#### TOP K 20

In [426]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [427]:
avaliacaoRecall(True)

Duração: 2.029329
Recall: 0.5084745762711864


In [428]:
avaliacaoPrecision(True,20)

P@ 20: 0.025424 


In [429]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000550


In [430]:
avaliacaoRR(True)

RR: 0.294915


# Emenda + Indexacao

## 1- Sem pré processamento

In [431]:
indice = "sem_preprocessamento_emenda_indexacao"

## Avaliação

#### TOP K 5

In [432]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [433]:
avaliacaoRecall(False)

Duração: 5.003151
Recall: 0.43050847457627117


In [434]:
avaliacaoPrecision(False,5)

P@ 5: 0.086102 


In [435]:
avaliacaoMAP(False,5)

MAP@ 5: 0.001758


In [436]:
avaliacaoRR(False)

RR: 0.244068


#### TOP K 10

In [437]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [438]:
avaliacaoRecall(False)

Duração: 4.319819
Recall: 0.5016949152542373


In [439]:
avaliacaoPrecision(False,10)

P@ 10: 0.050169 


In [440]:
avaliacaoMAP(False,10)

MAP@ 10: 0.001104


In [441]:
avaliacaoRR(False)

RR: 0.244068


#### TOP K 20

In [442]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [443]:
avaliacaoRecall(False)

Duração: 4.459987
Recall: 0.5491525423728814


In [444]:
avaliacaoPrecision(False,20)

P@ 20: 0.027458 


In [445]:
avaliacaoMAP(False,20)

MAP@ 20: 0.000621


In [446]:
avaliacaoRR(False)

RR: 0.244068


## 5- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword


In [447]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]
    terms = " ".join(terms)
    return terms

In [448]:
indice = "preprocessamento_5_emenda_indexacao"

## Avaliação

#### TOP K 5

In [449]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [450]:
avaliacaoRecall(True)

Duração: 2.754207
Recall: 0.4406779661016949


In [451]:
avaliacaoPrecision(True,5)

P@ 5: 0.088136 


In [452]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001782


In [453]:
avaliacaoRR(True)

RR: 0.261017


#### TOP K 10

In [454]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [455]:
avaliacaoRecall(True)

Duração: 1.674593
Recall: 0.5220338983050847


In [456]:
avaliacaoPrecision(True,10)

P@ 10: 0.052203 


In [457]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001042


In [458]:
avaliacaoRR(True)

RR: 0.261017


#### TOP K 20

In [459]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [460]:
avaliacaoRecall(True)

Duração: 1.803471
Recall: 0.5898305084745763


In [461]:
avaliacaoPrecision(True,20)

P@ 20: 0.029492 


In [462]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000659


In [463]:
avaliacaoRR(True)

RR: 0.261017


## 9- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP)

In [464]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [465]:
indice = "preprocessamento_8_emenda_indexacao"

## Avaliação

#### TOP K 5

In [466]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [467]:
avaliacaoRecall(True)

Duração: 3.631899
Recall: 0.46779661016949153


In [468]:
avaliacaoPrecision(True,5)

P@ 5: 0.093559 


In [469]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001999


In [470]:
avaliacaoRR(True)

RR: 0.305085


#### TOP K 10

In [471]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [472]:
avaliacaoRecall(True)

Duração: 2.524845
Recall: 0.5559322033898305


In [473]:
avaliacaoPrecision(True,10)

P@ 10: 0.055593 


In [474]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001300


In [475]:
avaliacaoRR(True)

RR: 0.305085


#### TOP K 20

In [476]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [477]:
avaliacaoRecall(True)

Duração: 2.801239
Recall: 0.6


In [478]:
avaliacaoPrecision(True,20)

P@ 20: 0.030000 


In [479]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000715


In [480]:
avaliacaoRR(True)

RR: 0.305085


## 11- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (Savoy)

In [481]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [482]:
indice = "preprocessamento_9_emenda_indexacao"

## Avaliação

#### TOP K 5

In [483]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [484]:
avaliacaoRecall(True)

Duração: 2.881931
Recall: 0.47796610169491527


In [485]:
avaliacaoPrecision(True,5)

P@ 5: 0.095593 


In [486]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002015


In [487]:
avaliacaoRR(True)

RR: 0.294915


#### TOP K 10

In [488]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [489]:
avaliacaoRecall(True)

Duração: 2.064242
Recall: 0.5457627118644067


In [490]:
avaliacaoPrecision(True,10)

P@ 10: 0.054576 


In [491]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001145


In [492]:
avaliacaoRR(True)

RR: 0.294915


#### TOP K 20

In [493]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [494]:
avaliacaoRecall(True)

Duração: 2.096312
Recall: 0.6


In [495]:
avaliacaoPrecision(True,20)

P@ 20: 0.030000 


In [496]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000698


In [497]:
avaliacaoRR(True)

RR: 0.294915


# Word n-gram + pré processamento básico + RSLP

## 20- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP) + unigram + bigram

In [498]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLPStemmer()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [499]:
indice = "preprocessamento_18_emenda_indexacao"

## Avaliação

#### TOP K 5

In [500]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [501]:
avaliacaoRecall(True)

Duração: 2.687335
Recall: 0.49491525423728816


In [502]:
avaliacaoPrecision(True,5)

P@ 5: 0.098983 


In [503]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002056


In [504]:
avaliacaoRR(True)

RR: 0.308475


#### TOP K 10

In [505]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [506]:
avaliacaoRecall(True)

Duração: 3.352003
Recall: 0.535593220338983


In [507]:
avaliacaoPrecision(True,10)

P@ 10: 0.053559 


In [508]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001138


In [509]:
avaliacaoRR(True)

RR: 0.308475


#### TOP K 20

In [510]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [511]:
avaliacaoRecall(True)

Duração: 3.399030
Recall: 0.5830508474576271


In [512]:
avaliacaoPrecision(True,20)

P@ 20: 0.029153 


In [513]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000694


In [514]:
avaliacaoRR(True)

RR: 0.308475


# Word n-gram + pré processamento básico + Savoy

## 26- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (Savoy) + unigram + bigram

In [515]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [516]:
indice = "preprocessamento_21_emenda_indexacao"

## Avaliação

#### TOP K 5

In [517]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [518]:
avaliacaoRecall(True)

Duração: 2.174809
Recall: 0.48135593220338985


In [519]:
avaliacaoPrecision(True,5)

P@ 5: 0.096271 


In [520]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002031


In [521]:
avaliacaoRR(True)

RR: 0.308475


#### TOP K 10

In [522]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [523]:
avaliacaoRecall(True)

Duração: 2.279386
Recall: 0.5322033898305085


In [524]:
avaliacaoPrecision(True,10)

P@ 10: 0.053220 


In [525]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001116


In [526]:
avaliacaoRR(True)

RR: 0.308475


#### TOP K 20

In [527]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [528]:
avaliacaoRecall(True)

Duração: 2.430792
Recall: 0.5830508474576271


In [529]:
avaliacaoPrecision(True,20)

P@ 20: 0.029153 


In [530]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000671


In [531]:
avaliacaoRR(True)

RR: 0.308475


## 22- Letra mínuscula + remoção de pontuação + remoção de acentuação e remoção de stopword + stemming (RSLP-S)

In [532]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
        
    return " ".join(terms)

In [533]:
indice = "preprocessamento_22_emenda_indexacao"

## Avaliação

#### TOP K 5

In [534]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [535]:
avaliacaoRecall(True)

Duração: 1.670764
Recall: 0.488135593220339


In [536]:
avaliacaoPrecision(True,5)

P@ 5: 0.097627 


In [537]:
avaliacaoMAP(True,5)

MAP@ 5: 0.002071


In [538]:
avaliacaoRR(True)

RR: 0.288136


#### TOP K 10

In [539]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [540]:
avaliacaoRecall(True)

Duração: 1.747652
Recall: 0.5627118644067797


In [541]:
avaliacaoPrecision(True,10)

P@ 10: 0.056271 


In [542]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001163


In [543]:
avaliacaoRR(True)

RR: 0.288136


#### TOP K 20

In [544]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [545]:
avaliacaoRecall(True)

Duração: 1.920639
Recall: 0.5932203389830508


In [546]:
avaliacaoPrecision(True,20)

P@ 20: 0.029661 


In [547]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000691


In [548]:
avaliacaoRR(True)

RR: 0.288136


## 23- Letra mínuscula + remoção de pontuação, acentuação e stopword + stemming (RSLP-S) + unigram + bigram

In [549]:
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

def preprocess(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    stemmer = RSLP_S()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    
    ngram = []
    ngram_1 = list(ngrams(terms, 1))
    ngram_2 = list(ngrams(terms, 2))
    for w in ngram_1:
        ngram.append(w[0])
        
    for w in ngram_2:
        string = w[0] + "_" + w[1]
        ngram.append(string)
    
    return " ".join(ngram)


In [550]:
indice = "preprocessamento_23_emenda_indexacao"

## Avaliação

#### TOP K 5

In [551]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=5)

In [552]:
avaliacaoRecall(True)

Duração: 2.305203
Recall: 0.47796610169491527


In [553]:
avaliacaoPrecision(True,5)

P@ 5: 0.095593 


In [554]:
avaliacaoMAP(True,5)

MAP@ 5: 0.001901


In [555]:
avaliacaoRR(True)

RR: 0.308475


#### TOP K 10

In [556]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=10)

In [557]:
avaliacaoRecall(True)

Duração: 2.172427
Recall: 0.5288135593220339


In [558]:
avaliacaoPrecision(True,10)

P@ 10: 0.052881 


In [559]:
avaliacaoMAP(True,10)

MAP@ 10: 0.001115


In [560]:
avaliacaoRR(True)

RR: 0.308475


#### TOP K 20

In [561]:
retriever = retrieve.Elastic(es=es,on=["content"],index=indice,key="name",k=20)

In [562]:
avaliacaoRecall(True)

Duração: 2.377127
Recall: 0.5796610169491525


In [563]:
avaliacaoPrecision(True,20)

P@ 20: 0.028983 


In [564]:
avaliacaoMAP(True,20)

MAP@ 20: 0.000671


In [565]:
avaliacaoRR(True)

RR: 0.308475
